# STRspy Config

This script creates the necessary config files to run STRspy on the Bioliquid Nanopore data.

# Load modules

In [1]:
import pandas as pd

# Individual BED file

Create one BED file per STR.

### Get reference genome

In [2]:
# Read in fasta file: remove line breaks and header
def read_fasta_genome(fasta_file,chromosome_header):
    clean_data = fasta_file.read().replace("\n", "")
    clean_data = clean_data.replace(chromosome_header,"") # get rid of header

    return clean_data

with open('../data/processed/chr17_selected.fa') as f: # update path if needed
    ref_genome = read_fasta_genome(f,'>chr17')

### Load Full list of STRs

In [4]:
# Load Full STR list
df = pd.read_csv('../data/raw/hg38.hipstr_reference.bed', sep='\t', header=None)
df.columns=['chr','start','end','repeats','NA','name','unit']

# Load STRspy test data
testdata = pd.read_csv('testset/testCustomDB/FGA.bed', sep='\t', header=None)
print('Data should look like this:')
testdata

FileNotFoundError: [Errno 2] No such file or directory: 'testset/testCustomDB/FGA.bed'

### Prepare data

In [ ]:
# Filter to Chr17 and locations 23M to 27M
df['chr']=df['chr'].str[3:]
# Filter chr17
df = df.loc[df['chr']=='17']
# Filter locations 23M to 27M
# df = df.loc[(df['start']>=23000000) & (df['end']<=27000000)]
# Get columns
df = df[['chr','start','end','name']]

In [ ]:
df.tail()

### Save each STR in different BED file for first 500

In [13]:
# Loop: create single STR files
# for n in range(len(df)):
for n in range(500):
    str_out = df.iloc[[n]]
    str_name = str_out['name'].values[0]
    str_out.to_csv(f"bioliquid-data/db/{str_name}.bed", header=False, index=False, sep='\t')
    
    myfasta = open(f"bioliquid-data/db/{str_name}.fa","w")
    start = df['start'].iloc[n]
    end = df['end'].iloc[n]
    # Extract reads
    padded_str=ref_genome[start-150:end+150]
    # Write to file
    myfasta.write('>')
    myfasta.write(str_name)
    myfasta.write('\n')
    myfasta.write(padded_str)
    myfasta.write('\n')
    myfasta.close()

# Older script

In [118]:
for str_number in range(len(df)):
    myfasta = open('bioliquid-data/str.fa','aw')
    # Get STR name, start and end
    str_name = df['name'].iloc[str_number]
    start = df['start'].iloc[str_number]
    end = df['end'].iloc[str_number]
    # Extract reads
    padded_str=ref_genome[start-150:end+150]

    # Write to file
    myfasta.write('>')
    myfasta.write(str_name)
    myfasta.write('\n')
    myfasta.write(padded_str)
    myfasta.write('\n')
    
myfasta.close()

out = df
out.to_csv('bioliquid-data/str.bed', header=False, index=False, sep='\t')

# Region BED file (all STRs)

In [112]:
out.to_csv('bioliquid-data/all_strs.bed', header=False, index=False, sep='\t')

# STR fasta file

Get STR +150bp -150bp

In [121]:
# Load test data to see what format we need
with open('testset/testCustomDB/FGA.fa', 'r') as f:
    text = f.read()
    print(text)

>FGA_[GGAA]2_GGAG_[AAAG]9__AA_AAAA_[GAAA]3_16.2
GGCTCAGAGGTGTGGTGATGTAATGGGCTTTGGAATTAGACAGGGACTGAACATTTGCTT
TTGAAATTTACTATCTATGTACCGTTGGAAAATTTACTTAATATCTCTGAATTTTTTTTC
TTCAACTGTGGAGTGAGGAAAATAATACCTACTTTTAGGTAGATGATGGATATAACACTT
TTCTCTGCATATAGTAGACACTCAGTGCATAACTATCGCCTTCCTTTTCCCTCTACTCAG
AAACAAGGACATCTGGGACCACAGCCACATACTTACCTCCAGTCGTTTCATATCAACCAA
CTGAGCTCTAACATTTTTCTGCAGAAGCTGGATATGCTGTACTTTTTCTATGACTTTGCG
CTTCAGGACTTCAATTCTGCTTCTCAGATCCTCTGACACTCGGTTGTAGGTATTATCACG
GTCTGAAATCGAAAATATGGTTATTGAAGTAGCTGCTGAGTGATTTGTCTGTAATTGCCA
GCAAAAAAGAAAGGAAGAAAGGAAGGAAGGAGAAAGAAAGAAAGAAAGAAAGAAAGAAAG
AAAGAAAGAAAAAAGAAAGAAAGAAACTAGCTTGTAAATATGCCTAATTTTATTTTGGTT
ACAGTTTAATCTGTGAGTTCAAAACCTATGGGGCATTTGACTTTTGGATAATGTTATGCC
CTGCAGCCTTCCATGAATGCCAGTTAAGATGTCCTAATAGCAATTAGTAATCCCAAAGAA
ATATAGAAGAAGAACTTTCTTTGGAATTTTAAAGGTGTAATTTGGAGTTAAAATAGTTGG
TTTGATTGCATTTCAATTATTTTATAACATCCTTAATCAAGGGACTTGAACATATTGGAT
TTTCTTACTGATGAGCTTTTCTTTTTAATCTATAGATTTGAAATGGTTCCTAAGCTGTTT
TGGGTCAACAGGATCACTCACTTGCCAGCTAGTGTTG

## Get STR

In [115]:
start = out['start'].values[0]
end = out['end'].values[0]

# Extract reads
padded_str=ref_genome[start-150:end+150]

# Extract name
str_name = out['name'].values[0]

## Save

In [116]:
df.shape

(58887, 4)

In [118]:
myfasta = open('bioliquid-data/str.fa','w')
myfasta.close()
myfasta = open('bioliquid-data/str.fa','a')

for str_number in range(len(df)):
    # Get STR name, start and end
    str_name = df['name'].iloc[str_number]
    start = df['start'].iloc[str_number]
    end = df['end'].iloc[str_number]
    # Extract reads
    padded_str=ref_genome[start-150:end+150]

    # Write to file
    myfasta.write('>')
    myfasta.write(str_name)
    myfasta.write('\n')
    myfasta.write(padded_str)
    myfasta.write('\n')
    
myfasta.close()

# All STRs

In [119]:
myfasta = open('bioliquid-data/all_strs.fa','w')
myfasta.close()
myfasta = open('bioliquid-data/all_strs.fa','a')

for str_number in range(len(df)):
    # Get STR name, start and end
    str_name = df['name'].iloc[str_number]
    start = df['start'].iloc[str_number]
    end = df['end'].iloc[str_number]
    # Extract reads
    padded_str=ref_genome[start-150:end+150]

    # Write to file
    myfasta.write('>')
    myfasta.write(str_name)
    myfasta.write('\n')
    myfasta.write(padded_str)
    myfasta.write('\n')
    
myfasta.close()

# Remove reads from BAM file

In [23]:
%%bash
samtools view -h ~/work/code/strspy/bioliquid-data/bioliquid_run1_chr17.bam | grep 505171f5-1f0b-4bb2-b855-5cd3d9ce7554 | samtools view -bS -o ~/work/code/strspy/bioliquid-data/chr17_filtered.bam -


[main_samview] fail to read the header from "-".


CalledProcessError: Command 'b'samtools view -h ~/work/code/strspy/bioliquid-data/bioliquid_run1_chr17.bam | grep 505171f5-1f0b-4bb2-b855-5cd3d9ce7554 | samtools view -bS -o ~/work/code/strspy/bioliquid-data/chr17_filtered.bam -\n'' returned non-zero exit status 1.